# Building RAG from Scratch (Open-source only!)


- Sentence Transformers as the embedding model
- Postgres as the vector store (we support many other [vector stores](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html) too!)
- Llama 2 as the LLM (through [llama.cpp](https://github.com/ggerganov/llama.cpp))

## Setup

We setup our open-source components.
1. Sentence Transformers
2. Llama 2
3. We initialize postgres and wrap it with our wrappers/abstractions.

#### Sentence Transformers

In [80]:
#%pip install llama-index-readers-file pymupdf
#%pip install llama-index-vector-stores-postgres
#%pip install llama-index-embeddings-huggingface
#%pip install llama-index-llms-llama-cpp

In [81]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

#### Llama CPP

In this notebook, we use the [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) model, along with the proper prompt formatting.

Check out our [Llama CPP guide](https://gpt-index.readthedocs.io/en/stable/examples/llm/llama_2_llama_cpp.html) for full setup instructions/details.

In [82]:
#!pip install llama-cpp-python

In [83]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/busraoguzoglu/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [5]:
#!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [84]:
import psycopg2

# Define your database connection parameters
db_name = "rag_db"  # Use the existing database name here
host = "localhost"
password = "password"  # Replace with your actual PostgreSQL password
port = "5432"  # Default PostgreSQL port
user = "myuser"  # Replace with your actual PostgreSQL username

# Connect directly to the 'rag_db' database
conn = psycopg2.connect(
    dbname=db_name,  # Connect directly to 'rag_db'
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# Optionally, perform any operations on 'rag_db' using a cursor
with conn.cursor() as c:
    # Drop and create operations are not needed if the database already exists
    # Here you can perform other database setup actions if necessary
    print(f"Connected to {db_name} successfully.")

# Close the connection when done
#conn.close()

Connected to rag_db successfully.


In [85]:
from sqlalchemy import make_url

from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database="rag_db",  # Replace with your actual database name
    host="localhost",   # Adjust if your database is hosted elsewhere
    password="password",  # Replace with your actual password
    port="5432",        # Default PostgreSQL port
    user="myuser",      # Replace with your actual user name
    table_name="llama2_paper",  # Adjust if you want a different table name
    embed_dim=384,  # Ensure this matches your embedding dimension / openai embedding dimension
)

## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

We fast-track the steps here (can skip metadata extraction). More details can be found [in our dedicated ingestion guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/ingestion.html).

## Load Data: Data is in 'data' folder

In [86]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [90]:
# Load all PDF documents from the folder and add metadata
folder_path = Path("./data")
documents = []

for pdf_file in folder_path.glob("*.pdf"):
    loader = PyMuPDFReader()
    loaded_docs = loader.load(file_path=str(pdf_file))
    print(f"{pdf_file.name}: {len(loaded_docs)} documents loaded")
    
    # Add metadata (e.g., filename) to each document
    for doc in loaded_docs:
        doc.metadata = {"source": pdf_file.name}
        documents.append(doc)


5.pdf: 10 documents loaded
4.pdf: 12 documents loaded
1.pdf: 18 documents loaded
3.pdf: 16 documents loaded
2.pdf: 10 documents loaded


In [91]:
print(documents[0])
print(len(documents))

Doc ID: 8e49ddfa-3d0a-48e4-898b-d070c783a848
Text: JNS JOURNAL OF NUTRITIONAL SCIENCE RESEARCH ARTICLE Workers’
healthy eating practices during the COVID-19 pandemic and their
relationship with physical activity and quality of life Alana do
Nascimento Oliveira , Lize Stangarlin-Fiori and Caroline Opolski
Medeiros* Postgraduate Program in Food and Nutrition, Federal
University of Parana, Curitiba...
66


### 2. Use a Text Splitter to Split Documents

In [92]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [99]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    #print(f"Document {doc_idx} has {len(cur_text_chunks)} chunks")
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))   


In [100]:
print(len(text_chunks))

114


### 3. Manually Construct Nodes from Text Chunks

In [101]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
        metadata={
            **documents[doc_idxs[idx]].metadata,  # This now includes 'source'
            "doc_index": doc_idxs[idx]  # Optional: document index
        }
    )
    nodes.append(node)

In [102]:
nodes[1]

TextNode(id_='494c12f7-d7d4-4446-b0b9-7fe3724fed3a', embedding=None, metadata={'source': '5.pdf', 'doc_index': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This change in the lifestyle of the population is partly, due to\nthe need for measures to contain the spread of COVID-19, such\nas social isolation. This has proven to be effective(8) and has led\nmany people to perform their work remotely.(9) This mobility\nrestriction has had direct effects on psychological factors, such\nas an increase in cases of anxiety and depression and a reduction\nin the practice of physical activities.(10–12) In addition, eating\nhabits were also inﬂuenced both by economic factors, due to the\nreduction in the population’s income, as well as by the\nconsumption of foods with higher energy density.(13,14)\nIn the period before the pandemic, the consumption of fresh\nand minimally processed foods represented approximately 70%\nof the total caloric intake by the

### 4. Generate Embeddings for each Node

Here we generate embeddings for each Node using a sentence_transformers model.

In [103]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="text")
    )
    node.embedding = node_embedding

In [104]:
print(type(nodes[0].embedding))
print(nodes[0].embedding)
print(len(nodes))

<class 'list'>
[-0.0033521901350468397, 0.023529022932052612, 0.031206179410219193, 0.011556013487279415, 0.04029195383191109, 0.057001177221536636, 0.019863374531269073, 0.002204005839303136, -0.001091738580726087, -0.008093681186437607, -0.018772142007946968, -0.06604456901550293, -0.0016525505343452096, 0.016629204154014587, -0.0016811549430713058, -0.029079772531986237, 0.007660300936549902, -0.024458175525069237, 0.0007334882975555956, 0.018140992149710655, -0.021003102883696556, -0.016628552228212357, -0.017473481595516205, -0.008948509581387043, 0.03527719900012016, 0.022544119507074356, -0.021890919655561447, -0.03420296311378479, -0.07296150177717209, -0.20341856777668, -0.021619297564029694, -0.04542139917612076, 0.0255585927516222, -0.005422128830105066, -0.06182954087853432, -0.0031680024694651365, 0.030594294890761375, 0.01684347353875637, -0.012634526006877422, -0.01120906975120306, 0.005322073586285114, 0.018391428515315056, 0.038591090589761734, -0.03580818325281143, -0

### 5. Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [105]:
vector_store.add(nodes)

['01c7e078-448c-4ff9-84f6-79bdc6372cdc',
 '494c12f7-d7d4-4446-b0b9-7fe3724fed3a',
 'd4b0ea40-ac22-4ea8-8d17-8ae05c6ae548',
 '54a316d5-80ba-4428-bf11-09a26e119e05',
 'f0fcbc1b-8ae9-48f5-97c3-32d3b5f38cc4',
 '3476677e-2c6c-4387-8b57-c95da592a51f',
 '2d4b01ee-e456-4adf-afe5-c4b87a3b4de0',
 'ec43f618-2a62-4d8e-9e34-7017c4d84cd8',
 '8c964a22-2497-49d8-b0ee-96ed160e924e',
 '868d327a-4e82-41f5-a8f5-357a48df0153',
 'da5ac7e3-94b7-4f62-beeb-6f345a5d25fa',
 '836e8e61-e5c5-4d4a-8279-44a1e15188db',
 'e8e41b3b-bd7e-4582-abaf-572315c2c92c',
 '6e107de9-cd2f-436e-ad3a-83ad36080e36',
 '78755f2d-a444-4132-89f0-224751f973af',
 'e410369d-64f0-44cc-a21b-d8297ac0c94c',
 'beee5b1b-7455-4b3e-8d4a-18c37d1ad9f6',
 'de09223d-d864-47f2-9d76-9a0a6f4032a5',
 '48eeeae4-3e8f-4236-b36d-b9a2f29477e1',
 '43a892e7-6f9e-4f74-9d90-2f1b94add211',
 'c739a3e9-bf54-4614-8b27-8a1d2ca91857',
 '35ef521f-c4ed-45d5-be72-c5c11437a4e7',
 'f5afbead-1bab-4e93-b907-d5d6a80df3db',
 '7bce11c8-4f7d-490d-9f82-f67ef1dca6fc',
 'ce7e3a62-a258-

## Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at our [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [106]:
# Example query
query_str = "Which individuals play a central role in promoting healthy eating"

### 1. Generate a Query Embedding

In [107]:
query_embedding = embed_model.get_query_embedding(query_str)

### 2. Query the Vector Database

In [108]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [109]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

Nutrients 2024, 16, 1365
9 of 18
“consume all macronutrients, vitamins and minerals” and “eat five servings of fruits and
vegetables a day” acquired significantly more importance in later years.
The analysis revealed that female and male participants shared similar perceptions,
although significant differences were found in the importance attributed to “respect the
physiological signals of hunger and satiety”, “eat according to the food pyramid” and
“consume all macronutrients, vitamins and minerals” (Table 1).
3.3. Perceptions of Barriers to Adopting a Healthy Diet
It is also crucial to understand perceived impediments to healthy eating (Figures 3 and 4).
Students of both degrees signaled three factors as the main barriers: “my family’s eating
habits”, “lack of time to buy food, cook it, and eat it”, and “my emotional states”. “Lack of
information about healthy eating” was judged to be of least importance, especially by HND
students (χ2 = 13.96, p = 0.007) (Figure 3). “The pleasure of

### 3. Parse Result into a Set of Nodes

In [110]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

### 4. Put into a Retriever

In [111]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [112]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

## Plug this into our RetrieverQueryEngine to synthesize a response

In [113]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [114]:
query_str = "Which individuals play a central role in promoting healthy eating?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   10407.34 ms
llama_print_timings:      sample time =       0.98 ms /    39 runs   (    0.03 ms per token, 39877.30 tokens per second)
llama_print_timings: prompt eval time =   49760.50 ms /  2329 tokens (   21.37 ms per token,    46.80 tokens per second)
llama_print_timings:        eval time =    4734.21 ms /    38 runs   (  124.58 ms per token,     8.03 tokens per second)
llama_print_timings:       total time =   54512.03 ms /  2367 tokens


In [115]:
print(str(response))

 Professionals in the fields of health and food, such as dietitians, food scientists and technologists, play a central role in promoting healthy eating.


In [116]:
print(response.source_nodes[0].get_content())

Nutrients 2024, 16, 1365
2 of 18
reaching 16% of the adult population in 2022 [4]. At the same time, nowadays, cardiovascular
diseases are the main cause of death globally, representing 32% of all global deaths [5]. Parallel
and paradoxically, thinness has become progressively valued and fatphobia, the discrimina-
tion and stigmatization against fat individuals, has become a phenomenon widely present
worldwide [6–8]. Finally, food crises with important health, social and economic international
impacts, such as the “mad cow”, have also increased concerns about food consumption [9].
Attempts to define what constitutes a healthy diet and provide dietary recommenda-
tions have been made in different scientific fields and organizations, including the World
Health Organization (WHO). In response to the increase in prevalence rates of morbidity
and mortality associated with chronic non-communicable diseases, in 2004 the WHO ap-
proved the Global Strategy on Diet, Physical Activity and Health,